In [1]:
import os
os.chdir("C:/Users/1/Desktop/TopMost")
import topmost
from topmost.data import download_dataset

device = "cuda" # or "cpu"
dataset_dir = "./data/NYT"
dataset_name=dataset_dir.split("/")[-1]

In [2]:
########################### Neural Topic Models ####################################
# dataset for neural topic models.
# For combinedTM, add contextual_embed=True.1
# dataset = topmost.data.BasicDataset(dataset_dir, read_labels=True, device=device, pretrained_WE=True)
dataset = topmost.data.BasicDataset(dataset_dir, read_labels=True, device=device)

train_size:  8254
test_size:  918
vocab_size:  10000
average length: 175.429


In [3]:
# create a model
# model = topmost.ProdLDA(dataset.vocab_size)
method = "ETM"
# model = topmost.ETM(dataset.vocab_size, pretrained_WE=dataset.pretrained_WE)
# model = topmost.DecTM(dataset.vocab_size)
# model = topmost.TSCTM(dataset.vocab_size)
# model = topmost.CombinedTM(dataset.vocab_size, dataset.contextual_embed_size)
# model = topmost.NSTM(dataset.vocab_size, pretrained_WE=dataset.pretrained_WE)
# model = topmost.ECRTM(dataset.vocab_size, pretrained_WE=dataset.pretrained_WE)

if method == "ETM":
    model = topmost.ETM(dataset.vocab_size, pretrained_WE=dataset.pretrained_WE)
elif method == "DecTM":
    model = topmost.DecTM(dataset.vocab_size)
elif method == "TSCTM":
    model = topmost.TSCTM(dataset.vocab_size)
elif method == "CombinedTM":
    model = topmost.CombinedTM(dataset.vocab_size, dataset.contextual_embed_size)
elif method == "NSTM":
    model = topmost.NSTM(dataset.vocab_size, pretrained_WE=dataset.pretrained_WE)
elif method == "ECRTM":
    model = topmost.ECRTM(dataset.vocab_size, pretrained_WE=dataset.pretrained_WE)
else:
    # 处理未定义的method情况
    raise ValueError(f"未支持的模型方法: {method}，请选择以下之一: ETM, DecTM, TSCTM, CombinedTM, NSTM, ECRTM")
    
model = model.to(device)

# create a trainer
trainer = topmost.BasicTrainer(model, dataset, verbose=True)

# train the model
top_words, train_theta = trainer.train()

100%|██████████| 200/200 [00:59<00:00,  3.36it/s]


Topic 0: minister prime north said korea nuclear putin south parliament country international government korean missile power
Topic 1: sanders mrs say said mueller bernie schmidt name musk never declined changed personal change comment
Topic 2: new said city data public country social based year online system already million announced plans
Topic 3: far even south risk india right side though less likely found majority significant many especially
Topic 4: deal legislation push efforts pressure pushed amazon promised proposal effort sweeping fight agreed regulators backed
Topic 5: israel israeli france paris blasio jerusalem jewish citizens palestinians group spanish arab gaza palestinian spain
Topic 6: company companies power use technology say industry google largest using electric emissions internet cars businesses
Topic 7: archived barbaro won history win victory events sept opioid distancing defeat winning presidency odds wikileaks
Topic 8: new york last according year said month w

In [4]:

"""
测试集主题模型推理并保存结果到CSV文件

"""

import pandas as pd
import numpy as np
import torch
import os
from typing import List, Tuple, Union
os.chdir("C:/Users/1/Desktop/TopMost")

# preprocess = Preprocess()
# new_bow=preprocess.parse(dataset.test, vocab=dataset.vocab)
# trainer.test(torch.as_tensor(new_bow.toarray(), device=device).float())

testtheta=trainer.test(torch.as_tensor(dataset.test_bow, device=device).float())
# testresults[1]
test_texts_dir = dataset_dir+"/test_texts.txt"

output_dir="C:/Users/1/Desktop/TopMost/LCLLMTM/NTM_results"
topic_ids= testtheta.argmax(1) 
# print(topic_ids)

# 1. 读取测试文本
print("正在读取测试文本...")
with open(test_texts_dir, 'r', encoding='utf-8') as f:
    test_texts = [line.strip() for line in f.readlines()]

# 3. 生成主题词列表
topic_words_list = []
for topic_id in topic_ids:
    topic_words_list.append(top_words[topic_id])

# 4. 创建DataFrame
data = {
    'ID': range(1, len(test_texts) + 1),  # 从1开始的序号
    'text': test_texts,
    'topic ID': topic_ids,
    'topic word list': topic_words_list
}

df = pd.DataFrame(data)

# 5. 创建输出目录并保存
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, dataset_name+"_"+method+"_testdata_inferred.csv")

df.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"✅ CSV文件已生成: {output_path}")
print(f"📊 处理了 {len(test_texts)} 条文本，发现 {len(set(topic_ids))} 个主题")


# len(testtheta)


正在读取测试文本...
✅ CSV文件已生成: C:/Users/1/Desktop/TopMost/LCLLMTM/NTM_results\NYT_ETM_testdata_inferred.csv
📊 处理了 918 条文本，发现 48 个主题


In [29]:
os.chdir("C:/Users/1/Desktop/TopMost/LCLLMTM/topic_evaluation")
from topic_eval_combine import TopicEvaluationRunner

topwords_list=[i.split(" ") for i in top_words]
texts_path=output_path
# texts_path="C:/Users/1/Desktop/TopMost/LCLLMTM/data4LCLLM/NYT_Dataset.csv"
api_key='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'
model="qwen/qwen3-235b-a22b:free"
output_file=os.path.join(output_dir,dataset_name+"_"+method+"_topics_eval.json")


config = {
    'texts_path': texts_path,
    'topics_path': "C:/Users/1/Desktop/TopMost/LCLLMTM/LLM_response_results/topic_analysis_NYT_sampled_deepseek_deepseek_chat_v3.1.json",  # 直接提供关键词列表
    'api_key': api_key,
    'model': model,
    'output_file': output_file,
}

runner = TopicEvaluationRunner(config)

# # Run metric evaluation and print results
results = runner.run_metric_evaluation()
runner.print_results(results)

# Run LLM evaluation
_ = runner.run_llm_evaluation()



Successfully loaded CSV with utf-8 encoding
topic_diversity: 0.9040
npmi_coherence_window: 0.4600
=== LLM主题评估开始 ===
使用模型: qwen/qwen3-235b-a22b:free
待评估主题数量: 50
API密钥已设置: sk-or-v1-960451...a37fb0335a3c

正在评估主题 1/50...
主题 1: ['Iran', 'Politics and Government', 'Ahmadinejad', 'Khamenei', 'Nuclear Weapons', 'Embargoes', 'International Relations']
🤖 正在调用LLM评估主题 1 (尝试 1/3)...
   关键词: Iran, Politics and Government, Ahmadinejad, Khamenei, Nuclear Weapons...
✅ LLM响应已接收，正在解析结果...
📊 主题 1 评估完成 - coherence: 5, conciseness: 4, informativity: 5

正在评估主题 2/50...
主题 2: ['China', 'Politics and Government', 'Human Rights', 'Censorship', 'International Relations', 'United States', 'Xi Jinping']
🤖 正在调用LLM评估主题 2 (尝试 1/3)...
   关键词: China, Politics and Government, Human Rights, Censorship, International Relations...
✅ LLM响应已接收，正在解析结果...
📊 主题 2 评估完成 - coherence: 3, conciseness: 3, informativity: 4

正在评估主题 3/50...
主题 3: ['Iraq', 'Iraq War', 'Defense and Military Forces', 'Terrorism', 'Maliki', 'Politics and Gov

In [ ]:
# ########################### test new documents ####################################
# import torch
# from topmost import Preprocess

# preprocess = Preprocess()

# new_docs = [
#     "This is a new document about space, including words like space, satellite, launch, orbit.",
#     "This is a new document about Microsoft Windows, including words like windows, files, dos."
# ]

# parsed_new_docs, new_bow = preprocess.parse(new_docs, vocab=dataset.vocab)
# new_theta = trainer.test(torch.as_tensor(new_bow.toarray(), device=device).float())

# print(new_theta.argmax(1))

parsing texts: 100%|██████████| 2/2 [00:00<00:00, 4088.02it/s]

[44 10]



/etc/miniconda3/envs/topmost2/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
